In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
from transformers import BertTokenizerFast, BertForTokenClassification
from sklearn.model_selection import train_test_split

train_df = pd.read_json("/content/drive/MyDrive/Colab Notebooks/pii_detection_ner_bert/train.json")

train_df.head()

,document,full_text,tokens,trailing_whitespace,labels
0,7,Design Thinking for innovation reflexion-Avril...,"[Design, Thinking, for, innovation, reflexion,...","[True, True, True, True, False, False, True, F...","[O, O, O, O, O, O, O, O, O, B-NAME_STUDENT, I-..."
1,10,Diego Estrada\n\nDesign Thinking Assignment\n\...,"[Diego, Estrada, \n\n, Design, Thinking, Assig...","[True, False, False, True, True, False, False,...","[B-NAME_STUDENT, I-NAME_STUDENT, O, O, O, O, O..."
2,16,Reporting process\n\nby Gilberto Gamboa\n\nCha...,"[Reporting, process, \n\n, by, Gilberto, Gambo...","[True, False, False, True, True, False, False,...","[O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O..."
3,20,Design Thinking for Innovation\n\nSindy Samaca...,"[Design, Thinking, for, Innovation, \n\n, Sind...","[True, True, True, False, False, True, False, ...","[O, O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT..."
4,56,Assignment: Visualization Reflection Submitt...,"[Assignment, :, , Visualization, , Reflecti...","[False, False, False, False, False, False, Fal...","[O, O, O, O, O, O, O, O, O, O, O, O, B-NAME_ST..."


In [3]:
train_df['tokens_len'] = train_df['tokens'].apply(lambda x: len(x))
train_df['labels_len'] = train_df['labels'].apply(lambda x: len(x))

train_df

,document,full_text,tokens,trailing_whitespace,labels,tokens_len,labels_len
0,7,Design Thinking for innovation reflexion-Avril...,"[Design, Thinking, for, innovation, reflexion,...","[True, True, True, True, False, False, True, F...","[O, O, O, O, O, O, O, O, O, B-NAME_STUDENT, I-...",753,753
1,10,Diego Estrada\n\nDesign Thinking Assignment\n\...,"[Diego, Estrada, \n\n, Design, Thinking, Assig...","[True, False, False, True, True, False, False,...","[B-NAME_STUDENT, I-NAME_STUDENT, O, O, O, O, O...",563,563
2,16,Reporting process\n\nby Gilberto Gamboa\n\nCha...,"[Reporting, process, \n\n, by, Gilberto, Gambo...","[True, False, False, True, True, False, False,...","[O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O...",729,729
3,20,Design Thinking for Innovation\n\nSindy Samaca...,"[Design, Thinking, for, Innovation, \n\n, Sind...","[True, True, True, False, False, True, False, ...","[O, O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT...",1071,1071
4,56,Assignment: Visualization Reflection Submitt...,"[Assignment, :, , Visualization, , Reflecti...","[False, False, False, False, False, False, Fal...","[O, O, O, O, O, O, O, O, O, O, O, O, B-NAME_ST...",1927,1927
...,...,...,...,...,...,...,...
6802,22678,EXAMPLE – JOURNEY MAP\n\nTHE CHALLENGE My w...,"[EXAMPLE, –, JOURNEY, MAP, \n\n, THE, CHALLENG...","[True, True, True, False, False, True, True, F...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",601,601
6803,22679,Why Mind Mapping?\n\nMind maps are graphical r...,"[Why, Mind, Mapping, ?, \n\n, Mind, maps, are,...","[True, True, False, False, False, True, True, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",433,433
6804,22681,"Challenge\n\nSo, a few months back, I had chos...","[Challenge, \n\n, So, ,, a, few, months, back,...","[False, False, False, True, True, True, True, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",633,633
6805,22684,Brainstorming\n\nChallenge & Selection\n\nBrai...,"[Brainstorming, \n\n, Challenge, &, Selection,...","[False, False, True, True, False, False, True,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",679,679


In [4]:
train_df.query('tokens_len != labels_len')

,document,full_text,tokens,trailing_whitespace,labels,tokens_len,labels_len


In [5]:
train_df.isnull().sum()

document               0
full_text              0
tokens                 0
trailing_whitespace    0
labels                 0
tokens_len             0
labels_len             0
dtype: int64

In [6]:
feature_count = train_df['labels'].apply(pd.Series).stack().value_counts()

feature_count

O                   4989794
B-NAME_STUDENT         1365
I-NAME_STUDENT         1096
B-URL_PERSONAL          110
B-ID_NUM                 78
B-EMAIL                  39
I-STREET_ADDRESS         20
I-PHONE_NUM              15
B-USERNAME                6
B-PHONE_NUM               6
B-STREET_ADDRESS          2
I-URL_PERSONAL            1
I-ID_NUM                  1
dtype: int64

In [7]:
feature_names = feature_count.index.tolist()

feature_names

['O',
 'B-NAME_STUDENT',
 'I-NAME_STUDENT',
 'B-URL_PERSONAL',
 'B-ID_NUM',
 'B-EMAIL',
 'I-STREET_ADDRESS',
 'I-PHONE_NUM',
 'B-USERNAME',
 'B-PHONE_NUM',
 'B-STREET_ADDRESS',
 'I-URL_PERSONAL',
 'I-ID_NUM']

In [8]:
label_mapping = {
    'O': 0,
    'B-NAME_STUDENT': 1,
    'I-NAME_STUDENT': 2,
    'B-URL_PERSONAL': 3,
    'B-ID_NUM': 4,
    'B-EMAIL': 5,
    'I-STREET_ADDRESS': 6,
    'I-PHONE_NUM': 7,
    'B-USERNAME': 8,
    'B-PHONE_NUM': 9,
    'B-STREET_ADDRESS': 10,
    'I-URL_PERSONAL': 11,
    'I-ID_NUM': 12
}

train_df['label_ids'] = train_df.labels.apply(lambda labels: [label_mapping[label] for label in labels])

train_df

,document,full_text,tokens,trailing_whitespace,labels,tokens_len,labels_len,label_ids
0,7,Design Thinking for innovation reflexion-Avril...,"[Design, Thinking, for, innovation, reflexion,...","[True, True, True, True, False, False, True, F...","[O, O, O, O, O, O, O, O, O, B-NAME_STUDENT, I-...",753,753,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, ..."
1,10,Diego Estrada\n\nDesign Thinking Assignment\n\...,"[Diego, Estrada, \n\n, Design, Thinking, Assig...","[True, False, False, True, True, False, False,...","[B-NAME_STUDENT, I-NAME_STUDENT, O, O, O, O, O...",563,563,"[1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,16,Reporting process\n\nby Gilberto Gamboa\n\nCha...,"[Reporting, process, \n\n, by, Gilberto, Gambo...","[True, False, False, True, True, False, False,...","[O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O...",729,729,"[0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,20,Design Thinking for Innovation\n\nSindy Samaca...,"[Design, Thinking, for, Innovation, \n\n, Sind...","[True, True, True, False, False, True, False, ...","[O, O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT...",1071,1071,"[0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,56,Assignment: Visualization Reflection Submitt...,"[Assignment, :, , Visualization, , Reflecti...","[False, False, False, False, False, False, Fal...","[O, O, O, O, O, O, O, O, O, O, O, O, B-NAME_ST...",1927,1927,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, ..."
...,...,...,...,...,...,...,...,...
6802,22678,EXAMPLE – JOURNEY MAP\n\nTHE CHALLENGE My w...,"[EXAMPLE, –, JOURNEY, MAP, \n\n, THE, CHALLENG...","[True, True, True, False, False, True, True, F...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",601,601,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
6803,22679,Why Mind Mapping?\n\nMind maps are graphical r...,"[Why, Mind, Mapping, ?, \n\n, Mind, maps, are,...","[True, True, False, False, False, True, True, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",433,433,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
6804,22681,"Challenge\n\nSo, a few months back, I had chos...","[Challenge, \n\n, So, ,, a, few, months, back,...","[False, False, False, True, True, True, True, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",633,633,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
6805,22684,Brainstorming\n\nChallenge & Selection\n\nBrai...,"[Brainstorming, \n\n, Challenge, &, Selection,...","[False, False, True, True, False, False, True,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",679,679,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [9]:
model_name = "bert-base-uncased"

tokenizer = BertTokenizerFast.from_pretrained(model_name)

tokenizer

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


BertTokenizerFast(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [10]:
tokenizer.model_max_length

512

In [11]:
model_max_length = tokenizer.model_max_length

window_size = model_max_length
overlap = model_max_length / 4

split_tokens = []
split_label_ids = []
split_document_ids = []

for index, row in train_df.iterrows():
    tokens = row['tokens']
    label_ids = row['label_ids']
    document_id = row['document']

    for i in range(0, len(tokens), window_size):
        split_tokens.append(tokens[i:i+window_size])
        split_label_ids.append(label_ids[i:i+window_size])
        split_document_ids.append(document_id)

split_token_label_df = pd.DataFrame({
    'document': split_document_ids,
    'tokens': split_tokens,
    'label_ids': split_label_ids
})

split_token_label_df

,document,tokens,label_ids
0,7,"[Design, Thinking, for, innovation, reflexion,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, ..."
1,7,"[images, and, interconnections, ., This, secon...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,10,"[Diego, Estrada, \n\n, Design, Thinking, Assig...","[1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,10,"[., As, a, , company, we, also, learnt, that,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,16,"[Reporting, process, \n\n, by, Gilberto, Gambo...","[0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...
13145,22681,"[\n\n, After, the, presentation, ,, we, saw, t...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
13146,22684,"[Brainstorming, \n\n, Challenge, &, Selection,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
13147,22684,"[can, stand, fresh, and, original, against, wh...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
13148,22687,"[Mind, Mapping, \n\n, Challenge, \n\n, My, con...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [12]:
print(split_token_label_df['tokens'][0])

['Design', 'Thinking', 'for', 'innovation', 'reflexion', '-', 'Avril', '2021', '-', 'Nathalie', 'Sylla', '\n\n', 'Challenge', '&', 'selection', '\n\n', 'The', 'tool', 'I', 'use', 'to', 'help', 'all', 'stakeholders', 'finding', 'their', 'way', 'through', 'the', 'complexity', 'of', 'a', 'project', 'is', 'the', ' ', 'mind', 'map', '.', '\n\n', 'What', 'exactly', 'is', 'a', 'mind', 'map', '?', 'According', 'to', 'the', 'definition', 'of', 'Buzan', 'T.', 'and', 'Buzan', 'B.', '(', '1999', ',', 'Dessine', '-', 'moi', ' ', "l'intelligence", '.', 'Paris', ':', 'Les', 'Éditions', "d'Organisation", '.', ')', ',', 'the', 'mind', 'map', '(', 'or', 'heuristic', 'diagram', ')', 'is', 'a', 'graphic', ' ', 'representation', 'technique', 'that', 'follows', 'the', 'natural', 'functioning', 'of', 'the', 'mind', 'and', 'allows', 'the', 'brain', "'s", ' ', 'potential', 'to', 'be', 'released', '.', 'Cf', 'Annex1', '\n\n', 'This', 'tool', 'has', 'many', 'advantages', ':', '\n\n', '•', ' ', 'It', 'is', 'acces

In [13]:
max_seq_length = split_token_label_df['tokens'].apply(lambda x: len(x)).max()

tokenized_input = split_token_label_df['tokens'].apply(lambda tokens: tokenizer(tokens, max_length=max_seq_length, padding='max_length', truncation=True, is_split_into_words=True))

tokenized_input

0        [input_ids, token_type_ids, attention_mask]
1        [input_ids, token_type_ids, attention_mask]
2        [input_ids, token_type_ids, attention_mask]
3        [input_ids, token_type_ids, attention_mask]
4        [input_ids, token_type_ids, attention_mask]
                            ...                     
13145    [input_ids, token_type_ids, attention_mask]
13146    [input_ids, token_type_ids, attention_mask]
13147    [input_ids, token_type_ids, attention_mask]
13148    [input_ids, token_type_ids, attention_mask]
13149    [input_ids, token_type_ids, attention_mask]
Name: tokens, Length: 13150, dtype: object

In [14]:
tokenized_input[0].keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [15]:
print(tokenized_input[0]['input_ids'])

[101, 2640, 3241, 2005, 8144, 22259, 3258, 1011, 20704, 15928, 25682, 1011, 14085, 8865, 2666, 25353, 4571, 4119, 1004, 4989, 1996, 6994, 1045, 2224, 2000, 2393, 2035, 22859, 4531, 2037, 2126, 2083, 1996, 11619, 1997, 1037, 2622, 2003, 1996, 2568, 4949, 1012, 2054, 3599, 2003, 1037, 2568, 4949, 1029, 2429, 2000, 1996, 6210, 1997, 20934, 13471, 1056, 1012, 1998, 20934, 13471, 1038, 1012, 1006, 2639, 1010, 4078, 11493, 2063, 1011, 25175, 1048, 1005, 4454, 1012, 3000, 1024, 4649, 6572, 1040, 1005, 5502, 1012, 1007, 1010, 1996, 2568, 4949, 1006, 2030, 2002, 9496, 10074, 16403, 1007, 2003, 1037, 8425, 6630, 6028, 2008, 4076, 1996, 3019, 12285, 1997, 1996, 2568, 1998, 4473, 1996, 4167, 1005, 1055, 4022, 2000, 2022, 2207, 1012, 12935, 17827, 2487, 2023, 6994, 2038, 2116, 12637, 1024, 1528, 2009, 2003, 7801, 2000, 2035, 1998, 2515, 2025, 5478, 3278, 3430, 5211, 1998, 2064, 2022, 2589, 2855, 1528, 2009, 2003, 26743, 3468, 1528, 2009, 4473, 4937, 20265, 26910, 1998, 11383, 1997, 2592, 1528, 2009

In [16]:
print(tokenized_input[0].tokens())

['[CLS]', 'design', 'thinking', 'for', 'innovation', 'reflex', '##ion', '-', 'av', '##ril', '2021', '-', 'nat', '##hal', '##ie', 'sy', '##lla', 'challenge', '&', 'selection', 'the', 'tool', 'i', 'use', 'to', 'help', 'all', 'stakeholders', 'finding', 'their', 'way', 'through', 'the', 'complexity', 'of', 'a', 'project', 'is', 'the', 'mind', 'map', '.', 'what', 'exactly', 'is', 'a', 'mind', 'map', '?', 'according', 'to', 'the', 'definition', 'of', 'bu', '##zan', 't', '.', 'and', 'bu', '##zan', 'b', '.', '(', '1999', ',', 'des', '##sin', '##e', '-', 'moi', 'l', "'", 'intelligence', '.', 'paris', ':', 'les', 'editions', 'd', "'", 'organisation', '.', ')', ',', 'the', 'mind', 'map', '(', 'or', 'he', '##uri', '##stic', 'diagram', ')', 'is', 'a', 'graphic', 'representation', 'technique', 'that', 'follows', 'the', 'natural', 'functioning', 'of', 'the', 'mind', 'and', 'allows', 'the', 'brain', "'", 's', 'potential', 'to', 'be', 'released', '.', 'cf', 'annex', '##1', 'this', 'tool', 'has', 'many'

In [17]:
print(tokenized_input[0].word_ids())

[None, 0, 1, 2, 3, 4, 4, 5, 6, 6, 7, 8, 9, 9, 9, 10, 10, 12, 13, 14, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 36, 37, 38, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 52, 53, 53, 54, 55, 55, 56, 56, 57, 58, 59, 60, 60, 60, 61, 62, 64, 64, 64, 65, 66, 67, 68, 69, 70, 70, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 79, 79, 80, 81, 82, 83, 84, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 100, 102, 103, 104, 105, 106, 107, 108, 108, 110, 111, 112, 113, 114, 115, 117, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 136, 138, 140, 141, 142, 142, 144, 146, 147, 148, 148, 148, 149, 150, 151, 152, 154, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 166, 167, 168, 169, 170, 171, 172, 173, 174, 176, 177, 179, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 193, 195, 196, 197, 198, 199, 200, 202, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 214, 214, 214, 216, 218, 219, 219, 219, 221, 223

In [18]:
tokenized_word_ids = tokenized_input.apply(lambda x: x.word_ids())

tokenized_word_ids

0        [None, 0, 1, 2, 3, 4, 4, 5, 6, 6, 7, 8, 9, 9, ...
1        [None, 0, 1, 2, 2, 2, 2, 3, 4, 5, 6, 7, 8, 10,...
2        [None, 0, 1, 3, 4, 5, 7, 7, 8, 10, 11, 12, 14,...
3        [None, 0, 1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, ...
4        [None, 0, 1, 3, 4, 4, 5, 5, 5, 7, 9, 10, 11, 1...
                               ...                        
13145    [None, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, ...
13146    [None, 0, 0, 0, 2, 3, 4, 6, 6, 6, 7, 8, 9, 10,...
13147    [None, 0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, ...
13148    [None, 0, 1, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13,...
13149    [None, 0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, ...
Name: tokens, Length: 13150, dtype: object

In [19]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id is None:
            new_labels.append(-100)
        elif word_id != current_word:
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
            current_word = word_id
        else:
            label = labels[word_id]
            new_labels.append(label)

    return new_labels

split_token_label_df['aligned_label_ids'] = split_token_label_df.apply(lambda row: align_labels_with_tokens(row['label_ids'], tokenized_word_ids.loc[row.name]), axis=1)

split_token_label_df

,document,tokens,label_ids,aligned_label_ids
0,7,"[Design, Thinking, for, innovation, reflexion,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, ...","[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, ..."
1,7,"[images, and, interconnections, ., This, secon...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,10,"[Diego, Estrada, \n\n, Design, Thinking, Assig...","[1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-100, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,10,"[., As, a, , company, we, also, learnt, that,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,16,"[Reporting, process, \n\n, by, Gilberto, Gambo...","[0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-100, 0, 0, 0, 1, 1, 2, 2, 2, 0, 0, 0, 0, 0, ..."
...,...,...,...,...
13145,22681,"[\n\n, After, the, presentation, ,, we, saw, t...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
13146,22684,"[Brainstorming, \n\n, Challenge, &, Selection,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
13147,22684,"[can, stand, fresh, and, original, against, wh...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
13148,22687,"[Mind, Mapping, \n\n, Challenge, \n\n, My, con...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [20]:
for token, label in zip(tokenizer.convert_ids_to_tokens(tokenized_input[0]['input_ids']), split_token_label_df['aligned_label_ids'][0]):
    print(f"{token} {label}")

[CLS] -100
design 0
thinking 0
for 0
innovation 0
reflex 0
##ion 0
- 0
av 0
##ril 0
2021 0
- 0
nat 1
##hal 1
##ie 1
sy 2
##lla 2
challenge 0
& 0
selection 0
the 0
tool 0
i 0
use 0
to 0
help 0
all 0
stakeholders 0
finding 0
their 0
way 0
through 0
the 0
complexity 0
of 0
a 0
project 0
is 0
the 0
mind 0
map 0
. 0
what 0
exactly 0
is 0
a 0
mind 0
map 0
? 0
according 0
to 0
the 0
definition 0
of 0
bu 0
##zan 0
t 0
. 0
and 0
bu 0
##zan 0
b 0
. 0
( 0
1999 0
, 0
des 0
##sin 0
##e 0
- 0
moi 0
l 0
' 0
intelligence 0
. 0
paris 0
: 0
les 0
editions 0
d 0
' 0
organisation 0
. 0
) 0
, 0
the 0
mind 0
map 0
( 0
or 0
he 0
##uri 0
##stic 0
diagram 0
) 0
is 0
a 0
graphic 0
representation 0
technique 0
that 0
follows 0
the 0
natural 0
functioning 0
of 0
the 0
mind 0
and 0
allows 0
the 0
brain 0
' 0
s 0
potential 0
to 0
be 0
released 0
. 0
cf 0
annex 0
##1 0
this 0
tool 0
has 0
many 0
advantages 0
: 0
• 0
it 0
is 0
accessible 0
to 0
all 0
and 0
does 0
not 0
require 0
significant 0
material 0
investment 0


In [21]:
split_token_label_df['input_ids'] = tokenized_input.apply(lambda x: x['input_ids'])
split_token_label_df['token_type_ids'] = tokenized_input.apply(lambda x: x['token_type_ids'])
split_token_label_df['attention_mask'] = tokenized_input.apply(lambda x: x['attention_mask'])

split_token_label_df

,document,tokens,label_ids,aligned_label_ids,input_ids,token_type_ids,attention_mask
0,7,"[Design, Thinking, for, innovation, reflexion,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, ...","[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, ...","[101, 2640, 3241, 2005, 8144, 22259, 3258, 101...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,7,"[images, and, interconnections, ., This, secon...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[101, 4871, 1998, 6970, 8663, 2638, 22014, 101...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,10,"[Diego, Estrada, \n\n, Design, Thinking, Assig...","[1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-100, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[101, 5277, 26482, 2640, 3241, 8775, 5107, 398...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,10,"[., As, a, , company, we, also, learnt, that,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[101, 1012, 2004, 1037, 2194, 2057, 2036, 2021...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,16,"[Reporting, process, \n\n, by, Gilberto, Gambo...","[0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-100, 0, 0, 0, 1, 1, 2, 2, 2, 0, 0, 0, 0, 0, ...","[101, 7316, 2832, 2011, 7664, 2080, 11721, 133...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
...,...,...,...,...,...,...,...
13145,22681,"[\n\n, After, the, presentation, ,, we, saw, t...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[101, 2044, 1996, 8312, 1010, 2057, 2387, 2008...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
13146,22684,"[Brainstorming, \n\n, Challenge, &, Selection,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[101, 14332, 20654, 2075, 4119, 1004, 4989, 14...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
13147,22684,"[can, stand, fresh, and, original, against, wh...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[101, 2064, 3233, 4840, 1998, 2434, 2114, 2054...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
13148,22687,"[Mind, Mapping, \n\n, Challenge, \n\n, My, con...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[101, 2568, 12375, 4119, 2026, 10552, 13220, 1...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [22]:
columns_to_copy = ['input_ids', 'token_type_ids', 'attention_mask', 'aligned_label_ids']

token_df = pd.DataFrame(split_token_label_df[columns_to_copy])
token_df.rename(columns={'aligned_label_ids': 'labels'}, inplace=True)

token_df

,input_ids,token_type_ids,attention_mask,labels
0,"[101, 2640, 3241, 2005, 8144, 22259, 3258, 101...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, ..."
1,"[101, 4871, 1998, 6970, 8663, 2638, 22014, 101...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"[101, 5277, 26482, 2640, 3241, 8775, 5107, 398...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,"[101, 1012, 2004, 1037, 2194, 2057, 2036, 2021...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,"[101, 7316, 2832, 2011, 7664, 2080, 11721, 133...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 0, 0, 0, 1, 1, 2, 2, 2, 0, 0, 0, 0, 0, ..."
...,...,...,...,...
13145,"[101, 2044, 1996, 8312, 1010, 2057, 2387, 2008...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
13146,"[101, 14332, 20654, 2075, 4119, 1004, 4989, 14...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
13147,"[101, 2064, 3233, 4840, 1998, 2434, 2114, 2054...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
13148,"[101, 2568, 12375, 4119, 2026, 10552, 13220, 1...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [23]:
train_dataset, test_dataset = train_test_split(token_df, test_size=0.3, random_state=42)
validation_dataset, test_dataset = train_test_split(test_dataset, test_size=0.5, random_state=42)

In [24]:
len(train_dataset), len(validation_dataset), len(test_dataset)

(9205, 1972, 1973)

In [25]:
!pip install datasets

In [26]:
from datasets import Dataset, DatasetDict

features = ['input_ids', 'token_type_ids', 'attention_mask', 'labels']

final_dataset = DatasetDict({
    'train': Dataset.from_pandas(train_dataset[features]),
    'validation': Dataset.from_pandas(validation_dataset[features]),
    'test': Dataset.from_pandas(test_dataset[features]),
})

final_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels', '__index_level_0__'],
        num_rows: 9205
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels', '__index_level_0__'],
        num_rows: 1972
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels', '__index_level_0__'],
        num_rows: 1973
    })
})

In [27]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [28]:
batch = data_collator([final_dataset['train'][i] for i in range(20)])

batch

{'input_ids': tensor([[  101,  1997, 22859,  ...,     0,     0,     0],
        [  101,  4901,  4202,  ...,     0,     0,     0],
        [  101,  9185,  1516,  ...,     0,     0,     0],
        ...,
        [  101,  3784,  3006,  ...,     0,     0,     0],
        [  101,  1996,  3570,  ...,     0,     0,     0],
        [  101,  4119,  1004,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), '__index_level_0__': tensor([ 5350,  3649,  5006,  7497,  8673,  3399,  4436, 10289,  7414,  6984,
         1360,  3196,  6762,  2037,  3282,   486, 

In [29]:
!pip install evaluate
!pip install seqeval

In [30]:
import evaluate
metric = evaluate.load("seqeval")

In [31]:
import numpy as np

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    true_labels = [[feature_names[l] for l in label if l!=-100] for label in labels]
    true_predictions = [[feature_names[p] for p, l in zip(prediction, label) if l!=-100] for prediction, label in zip(predictions, labels)]

    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)

    return {"precision": all_metrics['overall_precision'],
           "recall": all_metrics['overall_recall'],
           "f1": all_metrics['overall_f1'],
           "accuracy": all_metrics['overall_accuracy']}

In [32]:
bert_model = BertForTokenClassification.from_pretrained(model_name, num_labels=len(feature_count))

bert_model.config.num_labels

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


13

In [33]:
!pip install transformers[torch]

In [34]:
from transformers import TrainingArguments

output_directory = "/content/drive/MyDrive/Colab Notebooks/pii_detection_ner_bert"

args = TrainingArguments(output_dir=output_directory,
                         evaluation_strategy='epoch',
                         save_strategy='epoch',
                         learning_rate=2e-5,
                         num_train_epochs=3,
                         weight_decay=0.01,
                        report_to='none',
                        save_total_limit=1)

In [35]:
from transformers import Trainer

trainer = Trainer(model=bert_model,
                  args=args,
                  train_dataset= final_dataset['train'],
                  eval_dataset= final_dataset['validation'],
                  data_collator=data_collator,
                  compute_metrics=compute_metrics,
                  tokenizer=tokenizer)

trainer.train()

Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.003100,0.001540,0.822667,0.782995,0.802341,0.999586
2,0.001600,0.000983,0.891817,0.815990,0.852220,0.999651
3,0.000700,0.000947,0.845869,0.870558,0.858036,0.999671


/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-

TrainOutput(global_step=3453, training_loss=0.005857633760691448, metrics={'train_runtime': 2916.2659, 'train_samples_per_second': 9.469, 'train_steps_per_second': 1.184, 'total_flos': 7216427541980160.0, 'train_loss': 0.005857633760691448, 'epoch': 3.0})

In [36]:
reverse_label_mapping = {
    'LABEL_0': 'O',
    'LABEL_1': 'B-NAME_STUDENT',
    'LABEL_2': 'I-NAME_STUDENT',
    'LABEL_3': 'B-URL_PERSONAL',
    'LABEL_4': 'B-ID_NUM',
    'LABEL_5': 'B-EMAIL',
    'LABEL_6': 'I-STREET_ADDRESS',
    'LABEL_7': 'I-PHONE_NUM',
    'LABEL_8': 'B-USERNAME',
    'LABEL_9': 'B-PHONE_NUM',
    'LABEL_10': 'B-STREET_ADDRESS',
    'LABEL_11': 'I-URL_PERSONAL',
    'LABEL_12': 'I-ID_NUM'
}


In [38]:
test_df = pd.read_json("/content/drive/MyDrive/Colab Notebooks/pii_detection_ner_bert/test.json")

test_df.head()

,document,full_text,tokens,trailing_whitespace
0,7,Design Thinking for innovation reflexion-Avril...,"[Design, Thinking, for, innovation, reflexion,...","[True, True, True, True, False, False, True, F..."
1,10,Diego Estrada\n\nDesign Thinking Assignment\n\...,"[Diego, Estrada, \n\n, Design, Thinking, Assig...","[True, False, False, True, True, False, False,..."
2,16,Reporting process\n\nby Gilberto Gamboa\n\nCha...,"[Reporting, process, \n\n, by, Gilberto, Gambo...","[True, False, False, True, True, False, False,..."
3,20,Design Thinking for Innovation\n\nSindy Samaca...,"[Design, Thinking, for, Innovation, \n\n, Sind...","[True, True, True, False, False, True, False, ..."
4,56,Assignment: Visualization Reflection Submitt...,"[Assignment, :, , Visualization, , Reflecti...","[False, False, False, False, False, False, Fal..."


In [39]:
len(test_df)

10

In [54]:
trainer.state.global_step

3453

In [55]:
last_checkpoint_folder  = f"/content/drive/MyDrive/Colab Notebooks/pii_detection_ner_bert//checkpoint-{trainer.state.global_step}"

last_checkpoint_folder

'/content/drive/MyDrive/Colab Notebooks/pii_detection_ner_bert//checkpoint-3453'

In [56]:
from transformers import pipeline

token_classifier = pipeline("token-classification", model=last_checkpoint_folder, aggregation_strategy="simple")

In [57]:
test_df['full_text'][0]

"Design Thinking for innovation reflexion-Avril 2021-Nathalie Sylla\n\nChallenge & selection\n\nThe tool I use to help all stakeholders finding their way through the complexity of a project is the  mind map.\n\nWhat exactly is a mind map? According to the definition of Buzan T. and Buzan B. (1999, Dessine-moi  l'intelligence. Paris: Les Éditions d'Organisation.), the mind map (or heuristic diagram) is a graphic  representation technique that follows the natural functioning of the mind and allows the brain's  potential to be released. Cf Annex1\n\nThis tool has many advantages:\n\n•  It is accessible to all and does not require significant material investment and can be done  quickly\n\n•  It is scalable\n\n•  It allows categorization and linking of information\n\n•  It can be applied to any type of situation: notetaking, problem solving, analysis, creation of  new ideas\n\n•  It is suitable for all people and is easy to learn\n\n•  It is fun and encourages exchanges\n\n•  It makes visi

In [58]:
token_classifier(test_df['full_text'][0])

[{'entity_group': 'LABEL_0',
  'score': 0.9989931,
  'word': 'design thinking for innovation reflexion - avril 2021 -',
  'start': 0,
  'end': 52},
 {'entity_group': 'LABEL_1',
  'score': 0.9530471,
  'word': 'nathalie',
  'start': 52,
  'end': 60},
 {'entity_group': 'LABEL_2',
  'score': 0.85684067,
  'word': 'sylla',
  'start': 61,
  'end': 66},
 {'entity_group': 'LABEL_0',
  'score': 0.99990964,
  'word': "challenge & selection the tool i use to help all stakeholders finding their way through the complexity of a project is the mind map. what exactly is a mind map? according to the definition of buzan t. and buzan b. ( 1999, dessine - moi l ' intelligence. paris : les editions d ' organisation. ), the mind map ( or heuristic diagram ) is a graphic representation technique that follows the natural functioning of the mind and allows the brain ' s potential to be released. cf annex1 this tool has many advantages : • it is accessible to all and does not require significant material inves

In [60]:
import csv

file_name = "submission.csv"

with open(file_name, 'w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(['row_id', 'document', 'token', 'label'])

    for i in range(len(test_df)):
        pii_detection = token_classifier(test_df['full_text'][i])
        discovered_positions = []

        print(f"Now reading Row: {i}")

        for j in range(len(pii_detection)):
            if pii_detection[j]['entity_group'] != 'LABEL_0':
                pii_word = pii_detection[j]['word']
                pii_entity_group = pii_detection[j]['entity_group']

                tokens_list = [token.lower() for token in test_df['tokens'][i]]
                token_position = next((index for index, token in enumerate(tokens_list) if token == pii_word and index not in discovered_positions), None)

                if token_position is not None:
                    discovered_positions.append(token_position)
                    token = pii_word
                else:
                    token = ''

                row_id = i
                document = test_df['document'][i]
                token = token_position
                label = reverse_label_mapping[pii_entity_group]

                csv_writer.writerow([row_id, document, token, label])

                print(f"Word: {pii_word}, Predicted Label: {reverse_label_mapping[pii_entity_group]}")

    print("Finished Reading the Test Data")

Now reading Row: 0
Word: nathalie, Predicted Label: B-NAME_STUDENT
Word: sylla, Predicted Label: I-NAME_STUDENT
Word: nathalie, Predicted Label: B-NAME_STUDENT
Word: sylla, Predicted Label: I-NAME_STUDENT
Now reading Row: 1
Word: diego, Predicted Label: B-NAME_STUDENT
Word: estrada, Predicted Label: I-NAME_STUDENT
Word: diego, Predicted Label: B-NAME_STUDENT
Word: estrada, Predicted Label: I-NAME_STUDENT
Now reading Row: 2
Word: gilberto, Predicted Label: B-NAME_STUDENT
Word: gamboa, Predicted Label: I-NAME_STUDENT
Now reading Row: 3
Word: sindy, Predicted Label: B-NAME_STUDENT
Word: samaca, Predicted Label: I-NAME_STUDENT
Now reading Row: 4
Word: nadine, Predicted Label: B-NAME_STUDENT
Word: born, Predicted Label: I-NAME_STUDENT
Now reading Row: 5
Word: eladio, Predicted Label: B-NAME_STUDENT
Word: amaya, Predicted Label: I-NAME_STUDENT
Now reading Row: 6
Word: silvia, Predicted Label: B-NAME_STUDENT
Word: villalobos, Predicted Label: I-NAME_STUDENT
Now reading Row: 7
Word: sakir, Pre